In [1]:
# python libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import liana as li
import decoupler as dc

import session_info

In [2]:
import cell2cell as c2c

/nfs/team283/yd2/software/my_miniconda3/envs/liana/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
# import liana's rank_aggregate
from liana.mt import rank_aggregate

In [4]:
# import all individual methods
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Create a tensor and move it to the GPU
    tensor = torch.tensor([1.0, 2.0, 3.0]).cuda()
    print("Tensor on GPU:", tensor)
    
    # Perform a simple operation
    result = tensor * 2
    print("Result:", result)
else:
    print("CUDA is not available.")


In [ ]:
adata=sc.read_h5ad("/nfs/team283/yd2/WE_n3_scRNAseq_filt_cells_filt_genes_raw_counts_with_QC_and_annots_lvl5_20240404.h5ad")

In [ ]:
subtypes = ['meningeal fibroblast', 
            'mural PDGFRB+RGS5+',
            'dermal fibroblast FRZB+',
            'fibroblast progenitors PDGFRA+ZIC1+',
            'early endothelium',
            'mural PDGFRB+RGS5+KNCJ8+',
            'dermal fibroblast FRZB+FOXF1+',
            'dermal fibroblast FRZB+SIX2+',
            'hepatic stellate cell',
            'hepatocyte',
            'early endothelium liver',
            'megakaryocyte erythroid mast progenitor']

In [ ]:
sas = adata[adata.obs.cell_type_lvl5.isin(subtypes), :]

In [ ]:
# basic filters
sc.pp.filter_cells(sas, min_genes=200)
sc.pp.filter_genes(sas, min_cells=3)

In [ ]:
# filter low quality cells with standard QC metrics
sas.var['mt'] = sas.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(sas,
                           qc_vars=['mt'],
                           percent_top=None,
                           log1p=False,
                           inplace=True)
sas = sas[sas.obs.pct_counts_mt < 15, :]

In [ ]:
sas = sas[sas.obs.n_genes < 5500, :]

In [ ]:
# log1p normalize the data
sc.pp.normalize_total(sas)
sc.pp.log1p(sas)
sas.raw = sas

In [ ]:
li.mt.rank_aggregate.by_sample(sas,
                               sample_key='spatial_location',
                               groupby='cell_type_lvl5',
                               resource_name = 'consensus',
                               expr_prop=0.1, # must be expressed in expr_prop fraction of cells
                               min_cells = 5,
                               n_perms = 100,
                               use_raw = False, # run on log- and library-normalized counts
                               verbose = True,
                               inplace = True
                              )

In [ ]:
sorted_samples = sorted(sas.obs['spatial_location'].unique())

In [ ]:
# build the tensor
tensor = li.multi.to_tensor_c2c(liana_res=sas.uns['liana_res'], # LIANA's dataframe containing results
                                sample_key='spatial_location', # Column name of the samples
                                source_key='source', # Column name of the sender cells
                                target_key='target', # Column name of the receiver cells
                                ligand_key='ligand_complex', # Column name of the ligands
                                receptor_key='receptor_complex', # Column name of the receptors
                                score_key='magnitude_rank', # Column name of the communication scores to use
                                inverse_fun=lambda x: 1 - x, # Transformation function
                                how='outer', # What to include across all samples
                                outer_fraction=1/3., # Fraction of samples as threshold to include cells and LR pairs.
                                context_order=sorted_samples, # Order to store the contexts in the tensor
                               )

In [ ]:
context_dict = {x: x for x in sas.obs['spatial_location']}

In [33]:
dimensions_dict = [context_dict, None, None, None]
meta_tensor = c2c.tensor.generate_tensor_metadata(interaction_tensor=tensor,
                                                  metadata_dicts=dimensions_dict,
                                                  fill_with_order_elements=True
                                                 )

In [34]:
data_folder = '/lustre/scratch126/cellgen/team283/yd2/whole_embryo/11072024_tensor_c2c_SAS_liver/'
output_folder = '/lustre/scratch126/cellgen/team283/yd2/whole_embryo/11072024_tensor_c2c_SAS_liver/outputs'
c2c.io.directories.create_directory(data_folder)
c2c.io.directories.create_directory(output_folder)

/lustre/scratch126/cellgen/team283/yd2/whole_embryo/liana/ already exists.
/lustre/scratch126/cellgen/team283/yd2/whole_embryo/liana//outputs already exists.


In [35]:
c2c.io.export_variable_with_pickle(variable=tensor,
                                   filename=output_folder + '/Tensor.pkl')
c2c.io.export_variable_with_pickle(variable=meta_tensor,
                                   filename=output_folder + '/Tensor-Metadata.pkl')

/lustre/scratch126/cellgen/team283/yd2/whole_embryo/liana//outputs/Tensor.pkl  was correctly saved.
/lustre/scratch126/cellgen/team283/yd2/whole_embryo/liana//outputs/Tensor-Metadata.pkl  was correctly saved.


In [36]:
tensor = c2c.io.read_data.load_tensor(output_folder + '/Tensor.pkl')
meta_tensor = c2c.io.load_variable_with_pickle(output_folder + '/Tensor-Metadata.pkl')

In [ ]:
c2c.analysis.run_tensor_cell2cell_pipeline(tensor,
                                           meta_tensor,
                                           rank=None, # Number of factors to perform the factorization. If None, it is automatically determined by an elbow analysis
                                           tf_optimization='robust', # To define how robust we want the analysis to be.
                                           random_state=0, # Random seed for reproducibility
                                           device='cuda', # Device to use. If using GPU and PyTorch, use 'cuda'. For CPU use 'cpu'
                                           output_folder=output_folder, # Whether to save the figures in files. If so, a folder pathname must be passed
                                          )

Running Elbow Analysis


 40%|█████████████████████████████████████████████▌                                                                    | 10/25 [12:30<24:03, 96.22s/it]